In [53]:
from azureml.core import Workspace
ws=Workspace.from_config(path='./Azure Data Scientist')
print(ws.datastores.keys())

dict_keys(['azureml_globaldatasets', 'business_datastore', 'workspacefilestore', 'workspaceblobstore', 'workspaceworkingdirectory', 'workspaceartifactstore'])


In [54]:
from azureml.core.compute import ComputeTarget,AmlCompute
compute_config=AmlCompute.provisioning_configuration(vm_size='STANDARD_DS11_V2',
max_nodes=1,location='eastus2')
compute_target=ComputeTarget.create(workspace=ws,name='azurecomputecluster',provisioning_configuration=compute_config)
compute_target.wait_for_completion()

Provisioning operation finished, operation "Succeeded"


In [55]:
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment
myenv=Environment(name="My nb environment")
mydependencies=CondaDependencies.create(conda_packages=['pandas','scikit-learn','xgboost'])
myenv.python.conda_dependencies=mydependencies
myenv.register(ws)

{
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20220504.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "My nb environment",
    "python": {
        "baseCondaEnvironment": null,
        "condaDependencies": {
            "channels": [
                "anaconda",
                "conda-for

In [56]:
from azureml.core.runconfig import RunConfiguration
run_config=RunConfiguration()
run_config.target=compute_target
run_config.environment=myenv

In [57]:
from azureml.core import Dataset,Datastore
my_datastore=Datastore.get(workspace=ws,datastore_name='business_datastore')
my_dataset=Dataset.Tabular.from_delimited_files(path=(my_datastore,'./Loan Data/loan_prediction_updated.csv'))
my_dataset.register(workspace=ws,name="Loan Data NB")

{
  "source": [
    "('business_datastore', './Loan Data/loan_prediction_updated.csv')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ParseDelimited",
    "DropColumns",
    "SetColumnTypes"
  ],
  "registration": {
    "id": "ef2bdaf3-a31c-48c4-b6b3-afecca232459",
    "name": "Loan Data NB",
    "version": 1,
    "workspace": "Workspace.create(name='MLWorkspace', subscription_id='3291212d-f305-40a5-a60c-2bf6825d4013', resource_group='MachineLearningRG')"
  }
}

In [64]:
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import PipelineData
inputData=Dataset.get_by_name(workspace=ws, name='Loan Data NB')
outputDataFolder=PipelineData(name='outputdatafolder',
datastore=ws.get_default_datastore())
data_preparation_step=PythonScriptStep(script_name="loan_eda.py",
name='EDA STEP',
source_directory='.',
inputs=[inputData.as_named_input('raw_data')],
outputs=[outputDataFolder],
runconfig=run_config,
arguments=['--datafolder',outputDataFolder])

In [65]:
training_step=PythonScriptStep(source_directory='.',
name='XGBOOST PIPELINE',
inputs=[outputDataFolder],
runconfig=run_config,
script_name='xgboost_train.py',
arguments=['--datafolder',outputDataFolder]
)

In [66]:
from azureml.pipeline.core import Pipeline
my_pipeline_steps=[data_preparation_step,training_step]
my_pipeline= Pipeline(workspace=ws,steps=my_pipeline_steps)

In [68]:
from azureml.core import Experiment

new_experiment = Experiment(workspace=ws, name='pipeline-experiment')
new_pipeline_run = new_experiment.submit(my_pipeline)

new_pipeline_run.wait_for_completion(show_output=True)

Submitted PipelineRun 1f8be1f1-6c52-4023-b303-e606d7b73ebc
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/1f8be1f1-6c52-4023-b303-e606d7b73ebc?wsid=/subscriptions/3291212d-f305-40a5-a60c-2bf6825d4013/resourcegroups/MachineLearningRG/workspaces/MLWorkspace&tid=6bba74c5-e2b0-4111-98e8-d1056e7abd65
PipelineRunId: 1f8be1f1-6c52-4023-b303-e606d7b73ebc
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/1f8be1f1-6c52-4023-b303-e606d7b73ebc?wsid=/subscriptions/3291212d-f305-40a5-a60c-2bf6825d4013/resourcegroups/MachineLearningRG/workspaces/MLWorkspace&tid=6bba74c5-e2b0-4111-98e8-d1056e7abd65
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: b13dacd6-0e2b-4c12-b00c-7aee0e13f621
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/b13dacd6-0e2b-4c12-b00c-7aee0e13f621?wsid=/subscriptions/3291212d-f305-40a5-a60c-2bf6825d4013/resourcegroups/MachineLearningRG/workspaces/MLWorkspace&tid=6bba74c5-e2b0-4111-98e8-d1056e7abd65
StepRun( EDA

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "{\"NonCompliant\":\"Process '/azureml-envs/azureml_7747c226095ff3524f0e15cb2ba10e6f/bin/python' exited with code 1 and error message 'Execution failed. Process exited with status code 1. Error: Traceback (most recent call last):\\n  File \\\"loan_edaa.py\\\", line 44, in <module>\\n    non_numerical_columns.drop(columns=[\\\"Loan_Status\\\"], inplace=True)\\n  File \\\"/azureml-envs/azureml_7747c226095ff3524f0e15cb2ba10e6f/lib/python3.8/site-packages/pandas/util/_decorators.py\\\", line 311, in wrapper\\n    return func(*args, **kwargs)\\n  File \\\"/azureml-envs/azureml_7747c226095ff3524f0e15cb2ba10e6f/lib/python3.8/site-packages/pandas/core/frame.py\\\", line 4954, in drop\\n    return super().drop(\\n  File \\\"/azureml-envs/azureml_7747c226095ff3524f0e15cb2ba10e6f/lib/python3.8/site-packages/pandas/core/generic.py\\\", line 4267, in drop\\n 